In [ ]:
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D,BatchNormalization,GlobalAveragePooling3D
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop, SGD
from keras.layers.convolutional import MaxPooling3D, Conv3D
import cv2
import os
import random
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from collections import deque
from google.colab.patches import cv2_imshow

In [ ]:
datadir = "/content/drive/MyDrive/Colab Notebooks/3D-CNN/Dataset"
cat = ["non-violent", "Violent"]
training_data = []
for cate in cat:
    path = os.path.join(datadir,cate)
    class_num = cat.index(cate)
    
    for vid in os.listdir(path):
        path1 = os.path.join(path,vid)
        cap = cv2.VideoCapture(path1)
        frames = []
            
        for k in range(50):
            ret,frame = cap.read()
            frame = cv2.resize(frame,(224,224))
            frames.append(frame)
            #cv2.imshow("window",frame)
            cv2.waitKey(10)
                
        cap.release()
        cv2.destroyAllWindows()
        training_data.append([frames,class_num])

random.shuffle(training_data) # shuffling the dataset 
X = []
Y = []
for features,labels in training_data:
    X.append(features)
    Y.append(labels)
    
pickle_out = open("/content/drive/MyDrive/Colab Notebooks/3D-CNN/X_1.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("/content/drive/MyDrive/Colab Notebooks/3D-CNN/Y_1.pickle","wb")
pickle.dump(Y, pickle_out)
pickle_out.close()


In [ ]:
datadir = "/content/drive/MyDrive/Colab Notebooks/3D-CNN/Dataset"
cat = ["non-violent", "Violent"]
training_data = []
for cate in cat:
    path = os.path.join(datadir,cate)
    class_num = cat.index(cate)
    
    count = 0
    for vid in os.listdir(path):
        path1 = os.path.join(path,vid)
        count+=1
        if count == 10:
          break
        cap = cv2.VideoCapture(path1)
        frames = []
        
        
        while True:
            grabbed,frame = cap.read()
            if not grabbed:
                break
            frame = cv2.resize(frame,(224,224))
            
          
            frames.append(frame)
            cv2.waitKey(10)
              
        cap.release()
        training_data.append([frames,class_num])

random.shuffle(training_data) # shuffling the dataset 
X = []
Y = []
for features,labels in training_data:
    X.append(features)
    Y.append(labels)
    
pickle_out = open("C:\\Users\\snsha\\Desktop\\3DCNN\\X_1.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("C:\\Users\\snsha\\Desktop\\3DCNN\\Y_1.pickle","wb")
pickle.dump(Y, pickle_out)
pickle_out.close()

In [ ]:
if os.path.getsize("/content/drive/MyDrive/Colab Notebooks/3D-CNN/X_1.pickle") > 0:
    with open("/content/drive/MyDrive/Colab Notebooks/3D-CNN/X_1.pickle","rb") as f:
        unpickler = pickle.Unpickler(f)
        X = unpickler.load()
    with open("/content/drive/MyDrive/Colab Notebooks/3D-CNN/Y_1.pickle","rb") as f:
        unpickler = pickle.Unpickler(f)
        Y = unpickler.load()

#slpitting images into test and train sets
x_train_orig, x_test_orig, y_train_orig, y_test_orig = train_test_split(X, Y, test_size=0.15, random_state=1)
x_train = np.array(x_train_orig)
x_test = np.array(x_test_orig)
y_train = np.array(y_train_orig)
y_test = np.array(y_test_orig)



In [ ]:
x_train.shape

In [ ]:
columns = 4
plt.figsize = (10,10)
for i in range(16):
    plt.subplot(10/columns+1, columns, i+1)
    plt.imshow(x_train[85][i])

In [ ]:
lb = LabelBinarizer()
#y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)



In [ ]:
             
model = Sequential()
#1st layer 
model.add(Conv3D(filters = 16,kernel_size = (3,3,3), activation = "relu", padding = "same", name = "conv1", input_shape = (None,224,224,3)))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1,2,2), strides=(1,2,2),padding="valid", name="pool1"))
#1st layer 
model.add(Conv3D(filters = 64,kernel_size = (3,3,3), activation = "relu", padding = "same", name = "conv2"))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1,2,2), strides=(1,2,2),padding="valid", name="pool2"))
#2nd layer 
model.add(Conv3D(filters=128, kernel_size=(3,3,3), activation="relu", padding="same", name="conv3"))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size = (2,2,2), strides=(2,2,2), padding="valid", name="pool3"))
#3rd layer
model.add(Conv3D(filters=256, kernel_size=(3,3,3), activation="relu", padding="same", name = "conv4a"))
model.add(Conv3D(filters=256, kernel_size=(3,3,3), activation="relu", padding="same", name = "conv4b"))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding="valid", name="pool4"))
#4th layer
model.add(Conv3D(filters=512, kernel_size=(3,3,3), activation="relu", padding="same", name = "conv5a"))
model.add(Conv3D(filters=512, kernel_size=(3,3,3), activation="relu", padding="same", name = "conv5b"))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding="valid", name="pool5"))
#5th layer 
model.add(Conv3D(filters=512, kernel_size=(3,3,3), activation="relu", padding="same", name = "conv6a"))
model.add(Conv3D(filters=512, kernel_size=(3,3,3), activation="relu", padding="same", name = "conv6b"))
model.add(BatchNormalization())
model.add(ZeroPadding3D(padding=(0,1,1)))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding="valid", name="pool6"))

#model.add(Flatten())
model.add(GlobalAveragePooling3D())
#FC layers
    
model.add(Dense(512, activation='relu', name='fc9'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu', name='fc10'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(16, activation='relu', name='fc11'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
    
metrics = ["accuracy"]
optimizer = Adam(lr = 1e-5, decay = 1e-6)
model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=metrics)

    
    
    
    
    

In [ ]:
model.summary()

In [ ]:
hist = model.fit(x_train, y_train, validation_data=(x_test,y_test),batch_size=16,epochs=12,shuffle=True)

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/3D-CNN/3dcnn_3.h5")

In [ ]:
loaded_model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/3D-CNN/3dcnn_3.h5')

In [ ]:
X_test = x_test/255.0
loaded_model.evaluate(X_test, y_test)

In [ ]:
#Q = deque([],maxlen = 16)
vs = cv2.VideoCapture("/content/drive/MyDrive/Colab Notebooks/3D-CNN/test_video_1.mp4")
#vs = cv2.VideoCapture(0)

label = None
#output_path = 'result_3dcnn_1.mp4'

while True:
  inp = []
  for times in range(16):

    (grabbed, frame) = vs.read()
    #print(grabbed)
    if not grabbed:
      break 
   
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224,224)).astype("float32")
    frame = np.array(frame)
    frame = frame/255.0
    inp.append(frame)
    

  inp = np.array(inp)
  
  preds = loaded_model.predict(np.expand_dims(inp, axis=0))[0]  
  #preds = loaded_model.predict(input)
  #Q.append(preds)
    
  #results = np.array(Q).mean(axis=0)
  
  if preds >= 0.5:
    print("Violent")  
  else:
    print("non-violent")
      
  
  cv2_imshow(output)
  key = cv2.waitKey(1) & 0xFF
  if key == 27:
      break

cv2.destroyAllWindows() 